## Real numbers

Real numbers are represented as **floating point** numbers. A floating point number is stored in 3 pieces (sign bit, exponent, mantissa) so that every float is represetned as get +/- mantissa ^ exponent. Because of this, the interval between consecutive numbers is smallest (high precison) for numebrs close to 0 and largest for numbers close to the lower and upper bounds.

Because exponents have to be singed to represent both small and large numbers, but it is more convenint to use unsigned numbers here, the exponnent has an offset (also knwnn as the exponentn bias). For example, if the expoennt is an unsigned 8-bit number, it can rerpesent the range (0, 255). By using an offset of 128, it will now represent the range (-127, 128).

![float1](http://www.dspguide.com/graphics/F_4_2.gif)

**Note**: Intervals between consecutive floating point numbers are not constant. In particular, the precision for small numbers is much larger than for large numbers. In fact, approximately half of all floating point numbers lie between -1 and 1 when using the `double` type in C/C++ (also the default for `numpy`).

![float2](http://jasss.soc.surrey.ac.uk/9/4/4/fig1.jpg)

Because of this, if you are adding many numbers, it is more accurate to first add the small numbers before the large numbers.

#### IEEE 754 32-bit floating point representation

![img](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Float_example.svg/590px-Float_example.svg.png)

See [Wikipedia](https://en.wikipedia.org/wiki/Single-precision_floating-point_format) for how this binary number is evaluated to 0.15625.

In [118]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [119]:
from ctypes import c_int, c_float

In [120]:
s = c_int.from_buffer(c_float(-0.15625)).value
s = format(s, '032b')
s

'-1000001111000000000000000000000'

In [121]:
rep = {
    'sign': s[:1], 
    'exponent' : s[1:9:], 
    'fraction' : s[9:]
}
rep

{'sign': '-', 'exponent': '10000011', 'fraction': '11000000000000000000000'}

In [122]:
'%.20f' % (0.1 * 0.1 * 100)

'1.00000000000000022204'

### Never check for equality of floating point numbers

In [123]:
i = 0
loops = 0
while i != 1:
    i += 0.1 * 0.1
    loops += 1
    if loops == 1000000:
        break
i

10000.000000171856

In [124]:
i = 0
loops = 0
while np.abs(1 - i) > 1e-6:
    i += 0.1 * 0.1
    loops += 1
    if loops == 1000000:
        break
i

1.0000000000000007

### Associative law does not necessarily hold

In [125]:
6.022e23 - 6.022e23 + 1

1.0

In [126]:
1 + 6.022e23 - 6.022e23

0.0

### Distributive law does not hold

In [127]:
a = np.exp(1)
b = np.pi
c = np.sin(1)

In [128]:
a*(b+c)

10.82708950985241

In [129]:
a*b + a*c

10.827089509852408

### Catastrophic cancellation

Consider calculating sample variance

$$
s^2= \frac{1}{n(n-1)}\sum_{i=1}^n x_i^2 - (\sum_{i=1}^n x_i)^2
$$

Be  careful whenever you calculate the difference of potentially big numbers.

In [130]:
def var(x):
    """Returns variance of sample data using sum of squares formula."""
    
    n = len(x)
    return (1.0/(n*(n-1))*(n*np.sum(x**2) - (np.sum(x))**2))

### Numerically stable algorithms

#### What is the sample variance for numbers from a normal distribution with variance 1?

In [131]:
np.random.seed(15)
x_ = np.random.normal(0, 1, int(1e6))
x = 1e12 + x_
var(x)

-1328166901.4739892

In [132]:
np.var(x)

1.001345504504934

#### Underflow

We want to calculate the ration between the products of two sets of random numbers. Problems arise because the products are too small to be captured as a standard floating point number.

In [133]:
np.random.seed(4)
xs = np.random.random(1000)
ys = np.random.random(1000)
np.prod(xs), np.prod(ys), np.prod(xs)/np.prod(ys)

/var/folders/22/nmmy2bvn43dby4g5jhlr3jzw0000gn/T/ipykernel_72303/380396353.py:4: RuntimeWarning: invalid value encountered in scalar divide
  np.prod(xs), np.prod(ys), np.prod(xs)/np.prod(ys)


(0.0, 0.0, nan)

#### Prevent underflow by staying in log space

In [134]:
x = np.sum(np.log(xs))
y = np.sum(np.log(ys))
np.exp(x - y)

696432868222.2549

#### Overflow

Let's calculate

$$
\log(e^{1000} + e^{1000})
$$

Using basic algebra, we get the solution $\log(2) + 1000$.

In [135]:
x = np.array([1000, 1000])
np.log(np.sum(np.exp(x)))

/var/folders/22/nmmy2bvn43dby4g5jhlr3jzw0000gn/T/ipykernel_72303/2557198600.py:2: RuntimeWarning: overflow encountered in exp
  np.log(np.sum(np.exp(x)))


inf

In [136]:
np.logaddexp(*x)

1000.6931471805599

**logsumexp**

This function generalizes `logaddexp` to an arbitrary number of addends and is useful in a variety of statistical contexts.

Suppose we need to calculate a probability distribution $\pi$ parameterized by a vector $x$

$$
\pi_i = \frac{e^{x_i}}{\sum_{j=1}^n e^{x_j}}
$$

Taking logs, we get

$$
\log(\pi_i) = x_i - \log{\sum_{j=1}^n e^{x_j}}
$$

In [137]:
x = 1e6*np.random.random(100)

In [138]:
np.log(np.sum(np.exp(x))) 

/var/folders/22/nmmy2bvn43dby4g5jhlr3jzw0000gn/T/ipykernel_72303/1888342601.py:1: RuntimeWarning: overflow encountered in exp
  np.log(np.sum(np.exp(x)))


inf

In [139]:
from scipy.special import logsumexp

In [140]:
logsumexp(x)

989279.6334854055

### Other useful numerically stable functions 

**log1p and expm1**

In [141]:
np.exp(np.log(1 + 1e-8)) - 1

9.99999993922529e-09

In [142]:
np.expm1(np.log1p(1e-8))

1e-08